## AgglomerativeClustering Record Linkage pt2

In [2]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from scipy.spatial.distance import euclidean
import pandas as pd
from itertools import combinations

In [3]:
import warnings

# Ignora tutti i warning
warnings.filterwarnings("ignore")

# Codice che genera warning

# Riabilita i warning
warnings.filterwarnings("default")


## Test semplice

In [40]:
X = np.array([[1, 2], [1, 4], [1, 0],[4, 2],[4, 4],[4, 0],[1,2]])
clustering = AgglomerativeClustering(None,distance_threshold=1).fit(X)

In [41]:
clustering

AgglomerativeClustering(distance_threshold=1, n_clusters=None)

In [42]:
clustering.labels_

array([0, 4, 3, 5, 2, 1, 0])

## Scelta della threshold

In [4]:
def compute_threshold(elements):
    pairs = list(combinations(elements,2))
    # Calcola la distanza euclidea tra ciascuna coppia di vettori
    distances = [euclidean(pair[0], pair[1]) for pair in pairs]

    # Calcola la media delle distanze
    mean_distance = np.mean(distances)

    #print("Media distanza tra i vettori di embedding:", mean_distance)

    return mean_distance

## Caricamento dello schema mediato con cluster (Strategia Embedding Word2Vec)

In [5]:
schema_mediato = pd.read_csv("./../datasets/SchemaMediatoConCluster.csv")

/Users/alessandropesare/anaconda3/envs/FinalProjectIDD3_8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
schema_mediato
schema_mediato.drop('Unnamed: 0',axis = 1,inplace=True)

In [7]:
schema_mediato

,name,location,marketCap,sharePrice,category,foundationYear,block
0,Lacework,"United States, San Jose",$8.3B,NaN,Cybersecurity,2015,110
1,Tipalti,"United States, San Mateo",$8.3B,NaN,Fintech,2010,263
2,Tempus,"United States, Chicago",$8.1B,NaN,Health,2015,578
3,Anduril,"United States, Irvine",$8.48B,NaN,Artificial intelligence,2017,678
4,Bolt,"Estonia, Tallinn",$8.4B,NaN,Auto & transportation,2013,244
...,...,...,...,...,...,...,...
81701,PepsiCo,"Purchase, NY, United States",$241.05 Billion,NaN,NaN,NaN,0
81702,Costco,"Issaquah, WA, United States",$258.15 Billion,NaN,NaN,NaN,2133
81703,Broadcom,"San Jose, CA, United States",$265.27 Billion,NaN,NaN,NaN,90
81704,Coca-Cola,"Atlanta, GA, United States",$273.20 Billion,NaN,NaN,NaN,1158


In [8]:
num_clusters = schema_mediato['block'].nunique()
print(num_clusters)

2167


In [9]:
bert_vectors = pd.read_csv("./../datasets/MediatedSchemaVectors.csv")

## Clustering con vettori di embedding estratti tramite BERT

In [10]:
stats = {}

In [11]:
for i in range(0,num_clusters):
    print("Block corrente: ",i)
    records_cluster = schema_mediato[schema_mediato['block']==i]
    if(len(records_cluster)>1):
        vectors_cluster = bert_vectors.loc[records_cluster.index].copy() #lista degli indici di riga per il cluster corrente
        distance_threshold = compute_threshold(vectors_cluster.values.tolist())
        model = AgglomerativeClustering(n_clusters=None,distance_threshold=0.33*distance_threshold).fit(vectors_cluster)
        stats[i] = {}
        vectors_cluster['cluster'] = model.labels_
        for j in range(0,max(model.labels_)):
            stats[i][j] = vectors_cluster[vectors_cluster['cluster']==j].index
    else:
        stats[i] = {}
        stats[i][0] = records_cluster.index


Block corrente:  0
Block corrente:  1
Block corrente:  2
Block corrente:  3
Block corrente:  4
Block corrente:  5
Block corrente:  6
Block corrente:  7
Block corrente:  8
Block corrente:  9
Block corrente:  10
Block corrente:  11
Block corrente:  12
Block corrente:  13
Block corrente:  14
Block corrente:  15
Block corrente:  16
Block corrente:  17
Block corrente:  18
Block corrente:  19
Block corrente:  20
Block corrente:  21
Block corrente:  22
Block corrente:  23
Block corrente:  24
Block corrente:  25
Block corrente:  26
Block corrente:  27
Block corrente:  28
Block corrente:  29
Block corrente:  30
Block corrente:  31
Block corrente:  32
Block corrente:  33
Block corrente:  34
Block corrente:  35
Block corrente:  36
Block corrente:  37
Block corrente:  38
Block corrente:  39
Block corrente:  40
Block corrente:  41
Block corrente:  42
Block corrente:  43
Block corrente:  44
Block corrente:  45
Block corrente:  46
Block corrente:  47
Block corrente:  48
Block corrente:  49
Block corr

In [13]:
stats

{0: {0: Int64Index([71345, 71481], dtype='int64'),
  1: Int64Index([76656, 76663], dtype='int64'),
  2: Int64Index([76396, 76494], dtype='int64'),
  3: Int64Index([77938, 78001, 78072], dtype='int64'),
  4: Int64Index([20776, 20786, 20966], dtype='int64'),
  5: Int64Index([70994, 71376], dtype='int64'),
  6: Int64Index([76789, 76817], dtype='int64'),
  7: Int64Index([70955, 70977, 76264], dtype='int64'),
  8: Int64Index([81199, 81394, 81522], dtype='int64'),
  9: Int64Index([39134, 39223], dtype='int64'),
  10: Int64Index([19628, 23179], dtype='int64'),
  11: Int64Index([24911, 25703], dtype='int64'),
  12: Int64Index([71996, 72016], dtype='int64'),
  13: Int64Index([73602, 74049], dtype='int64'),
  14: Int64Index([67233, 67892], dtype='int64'),
  15: Int64Index([70582, 74581], dtype='int64'),
  16: Int64Index([80720, 80756], dtype='int64'),
  17: Int64Index([68404, 68443, 68468], dtype='int64'),
  18: Int64Index([76985, 77022], dtype='int64'),
  19: Int64Index([70599, 74591, 74594, 74

In [14]:
elementi_con_dimensione_maggiore_di_1 = sum(len(d) > 1 for d in stats.values())
elementi_con_dimensione_maggiore_di_1

2030

## Salvataggio su File

In [15]:
with open("./results_clustersW2V.txt", "w") as file:
    # Iterare sul dizionario e scrivere ogni coppia chiave-valore su una nuova riga
    for chiave, valore in stats.items():
        file.write(f"{chiave}: {valore}\n")